In [ ]:
from selenium import webdriver
import pandas as pd
import numpy as np
import time

# Login to fb @news page
def login(driver, account, password): 
    
    # Enter account
    driver.find_element_by_xpath("//*[@id=\"m_login_email\"]").send_keys(account)

    # Enter password
    driver.find_element_by_xpath("//*[@id=\"m_login_password\"]").send_keys(password)

    # Submit login
    driver.find_element_by_xpath("//*[@id=\"u_0_5\"]").click()
    time.sleep(2)
    
    # Proceed to home page
    driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/div/div[3]/div[1]/div/div/a').click()
    time.sleep(2)

# Get to article comment section
def toCommentSection(page_id, driver, article_id):
    # Generate url to each article comment section
    url_article = 'https://mobile.facebook.com/story.php?story_fbid=' + article_id + '&id=' + page_id + '&fs=5&focus_composer=0&ref=page_internal'
    driver.get(url_article)

def initCSV(page):
    import csv
    with open('results/'+page+'_posts.csv', 'w', encoding="utf-8") as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['article_index', 'headline', 'status', 'has_headline', 
                             'has_status', 'has_img', 'has_vid', 'likes'])
        
def appendResults(page, posts):
    df = pd.DataFrame(np.array(posts), columns=['article_index', 'headline', 'status', 'has_headline', 
                                                            'has_status', 'has_img', 'has_vid', 'likes'])
    
    # Append results to existing file
    with open('results/'+page+'_posts.csv', 'a', encoding="utf-8") as f:
        df.to_csv(f, header=False, index=False)

def getStatusLikes(driver, article_index):
    headline = ''
    status = ''
    has_headline = 0
    has_vid = 0
    has_img = 0
    has_status = 0
    likes = '0'
    
    try: 
        headline = driver.find_element_by_css_selector("span[data-ad-preview=\"headline\"]").text
        has_headline = 1
    except:
        headline = ''
        try:
            driver.find_element_by_css_selector("i[data-ad-preview=\"video-cover\"]")
            has_vid = 1
        except:
            pass
        try:
            driver.find_element_by_css_selector("i[data-sigil=\"photo-image\"]")
            has_img = 1
        except:
            pass
    
    try:
        status = driver.find_element_by_css_selector("div[data-ad-preview=\"message\"]").text
        has_status = 1
    except:
        pass
    
    try:
        likes = driver.find_element_by_css_selector("div[class=\"_1g06\"]").text
    except:
        pass
    
    res_tuple = (article_index, headline, status, has_headline, has_status, has_img, has_vid, likes)
    return res_tuple

def scrapePosts(page, page_id, driver, article_ids, start_idx): 
    initCSV(page)
    posts = []
    for i in range(start_idx,len(article_ids)):
        try:
            toCommentSection(page_id, driver, article_ids[i])
            posts.append(getStatusLikes(driver, i))
        except:
            pass
        
        # Save results for every 20 articles
        if (i%20 == 0) and (i!=0):
            try:
                appendResults(page, posts)
                print('up to article %i saved!' %i)
            except Exception as e:
                print('-------------------------------------------')
                print('Something went WRONG with articles %i to %i.' %(i-19, i))
                print(e)
                print('-------------------------------------------')
            # Clear results 
            posts = []
            
    # Save final remaining results
    if len(posts) != 0:
        try: 
            appendResults(page, posts)
            print('all results saved.')
        except Exception as e:
            print(e)
    
    print('all articles processed.')


def get(page, account, password, start_idx=0):
    # Start webdriver
    driver = webdriver.Chrome()
    driver.get('https://mobile.facebook.com')
    
    # Login to fb
    login(driver, account, password)
    
    # Set path to article_ids.csv
    path_to_article_ids = 'results-1000-win-fixed//'+page+'_ids.csv'
    
    # Set path to page_ids.csv
    path_to_page_ids = 'page_ids.csv'
    
    # Load article_ids
    article_ids = pd.read_csv(path_to_article_ids, header=None)[0][start_idx:]
    
    # Load page_ids
    page_ids = pd.read_csv(path_to_page_ids, header=0, index_col=0, dtype=str)
    page_id = page_ids.loc[page].get('page_id')
    
    # Start scraping, save in the process (every 20 articles)
    scrapePosts(page, page_id, driver, article_ids, start_idx)
